### Install AI Core Python SDK

In [1]:
# !pip install ai-core-sdk

In [1]:
import json
import requests
import time
import yaml
from IPython.display import clear_output
from pprint import pprint

from ai_core_sdk.models import ParameterBinding, Status

In [2]:
# Setup
aic_service_key_path = "./resources/aic_service_key.json"
git_setup_file_path = "./resources/git_setup.json"
docker_secret_file_path = "./resources/docker_secret.json"
env_file_path = "./resources/env.json"

### 1. Connect to your AI Core instance

In [3]:
# Load Library
with open(aic_service_key_path) as ask:
    aic_service_key = json.load(ask)

from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Create Connection
ai_core_client = AICoreV2Client(
    base_url = aic_service_key["serviceurls"]["AI_API_URL"] + "/v2", # The present AI API version is 2
    auth_url=  aic_service_key["url"] + "/oauth/token",
    client_id = aic_service_key["clientid"],
    client_secret = aic_service_key["clientsecret"]
)

*Returns no output*

### 2. Test connection

In [4]:
response = ai_core_client.repositories.query()
print(response.count) # Should return integer value else your values in above step are incorrect

0


### 2. Onboard the Git repository that contains the templates

In [5]:
# WARNING: Refrain from onboarding again if previously onboarded
#  else you will get AIAPIServerException 409

with open(git_setup_file_path) as gs:
		git_key = json.load(gs)

ai_core_client.repositories.create(
    name = "azure-openai-aicore",
    url = f"https://github.com/{git_key['username']}/azure-openai-aicore-cap-api", # Forked repo
    username = git_key["username"],
    password = git_key["password"]
)

AIAPIServerException: Failed to post /admin/repositories: A repository with name 'azure-openai-aicore' already exists 
 Status Code: 409, Request ID:None

In [14]:
# Check onboarded repositories
response = ai_core_client.repositories.query()
#
for repository in response.resources:
    print('Name:', repository.name)
    print('URL:', repository.url)
    print('Status:', repository.status)


*Expected Output*

```
...
Name: azure-openai-aicore
URL: https://github.com/john/azure-openai-aicore-cap-api
Status: RepositoryStatus.COMPLETED
```

### 3. Register an application

In [11]:
# WARNING: Run only once

ai_core_client.applications.create(
    repository_url = f"https://github.com/{git_key['username']}/azure-openai-aicore-cap-api",
    path = "01-ai-core-azure-openai-proxy/scenario", # Scan this folder for instruction YAML files
    revision = "HEAD"
)

In [12]:
# List scenarios scanned by the application created above
response = ai_core_client.scenario.query(resource_group='default')

for scenario in response.resources:
    print(scenario.__dict__)

{'id': 'azure-openai-proxy', 'name': 'Azure OpenAI Proxy', 'description': 'Azure OpenAI  Proxy', 'labels': None, 'created_at': datetime.datetime(2023, 10, 21, 17, 46, 41), 'modified_at': datetime.datetime(2023, 10, 21, 17, 46, 41)}


*Expected Output*

```
...
{'id': 'azure-openai-proxy', 'name': 'Azure OpenAI Proxy', 'description': 'Azure OpenAI  Proxy', 'labels': None, ...)}
```

### 4 Docker Hub (optional)
#### 4.1 Register Docker secret on SAP BTP, AI Core (optional)

In [13]:
with open(docker_secret_file_path) as dsf:
    docker_secret = json.load(dsf)


response = ai_core_client.docker_registry_secrets.create(
    name = docker_secret["name"],
    data = docker_secret["data"]
)

print(response.__dict__)

{'message': 'secret has been created'}


*Expected Output*

```
{'message': 'secret has been created'}
```

#### 4.2 Build and push Docker image (optional)
```
$ cd proxy
$ docker build -t {DOCKER_USERNAME}/azure-openai-proxy .
$ docker push {DOCKER_USERNAME}/azure-openai-proxy
```

THROUGH THE DOCKER CLI.  
See: https://developers.sap.com/tutorials/ai-core-aiapi-clientsdk-workflows.html#f824a41d-efe8-4883-8238-caef4ac5f789

### 5. Create a resource group (Optional)

In [16]:
# For Free Tier AI Core Serice:
#  you will not be able to create a new resource group.
#  resource group named `default` exists in all systems.
#  cell execution does not impact existing contents
#
# For paid AI Core service: 
#  IF you wish execute this step
#  You are NOT REQUIRED TO RE-EXECUTE or Redo previous completed steps.
#  Ensure that in the steps that follows modify with your resource group name

resource_group_id = "default" # For free tier; `default` exsits in all systems
# resource_group_id = "my-openai-proxy-ns" # For paid account you can create a namespace aka resource group
                                              # Other steps 

# response = ai_core_client.resource_groups.create(resource_group_id = resource_group_id)
# print(response.__dict__)

### 6. Create configuration to serve the model

In [17]:
with open(env_file_path) as efp:
    env_val = json.load(efp)

OPENAI_API_BASE = env_val["OPENAI_API_BASE"]
OPENAI_API_KEY = env_val["OPENAI_API_KEY"]
DOCKER_NAMESPACE = env_val["DOCKER_NAMESPACE"]

In [18]:
# No modification required in below snippet
response = ai_core_client.configuration.create(
    name = "azure-proxy-serve",
    scenario_id = "azure-openai-proxy",
    executable_id = "azure-openai-proxy",
    input_artifact_bindings = [],
    parameter_bindings = [
        ParameterBinding(key = "OPENAI_API_BASE", value = OPENAI_API_BASE),
        ParameterBinding(key = "OPENAI_API_KEY", value = OPENAI_API_KEY), 
        ParameterBinding(key = "DOCKER_NAMESPACE", value = DOCKER_NAMESPACE)
    ],
    resource_group = resource_group_id
)


serve_config_resp = response
print(response.__dict__)

{'id': '2205fc1d-3f83-431f-91e3-306103d82bc7', 'message': 'Configuration created'}


### 7. Actually serve the proxy

In [23]:
# Start proxy
response = ai_core_client.deployment.create(
    configuration_id=serve_config_resp.id,
    resource_group=resource_group_id
)

deployment_resp = response
print(response.__dict__)

{'id': 'de24078fdb34610e', 'message': 'Deployment scheduled.', 'deployment_url': '', 'status': <Status.UNKNOWN: 'UNKNOWN'>, 'ttl': None}


In [24]:
# Poll deployment status.
# No modification required in below snipet
status = None
while status != Status.RUNNING and status != Status.DEAD:
    time.sleep(5)
    clear_output(wait=True)
    # Get Status
    #
    deployment = response = ai_core_client.deployment.get(
        deployment_id=deployment_resp.id,
        resource_group=resource_group_id
    )
    status = deployment.status
    print("...... deployment status ......", flush=True)
    print(deployment.status)
    pprint(deployment.status_details)

    if deployment.status == Status.RUNNING:
        print(f"Deployment with {deployment_resp.id} complete!")

# Allow some time for deployment URL to get ready.
time.sleep(10)

KeyboardInterrupt: 

### 8. Do an inference request

In [ ]:
endpoint = f"{deployment.deployment_url}/v2/envs"
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           "ai-resource-group": resource_group_id,
           "Content-Type": "application/json"}
response = requests.get(endpoint, headers=headers)

legacy_davinci = False # set True if you have a davinci model deployment on Azure OpenAI Services
if legacy_davinci:
    body = {
        "engine": "<YOUR ENGINE>", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
                                   #   For information of deployment creation and name Refer article https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal
        "prompt": "Classify the following news article into 1 of the following categories: categories: [Business, Tech, Politics, Sport, Entertainment]\n\nnews article: Donna Steffensen Is Cooking Up a New Kind of Perfection. The Internet’s most beloved cooking guru has a buzzy new book and a fresh new perspective:\n\nClassified category:",
        "max_tokens": 60,
        "temperature": 0,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "top_p": 1,
        "best_of": 1,
        "stop": "null"
    }
    endpoint = f"{deployment.deployment_url}/v2/completion"
else:
    body = {
        "engine": "<YOUR ENGINE>", # include your engine from a deployment of an Azure OpenAI services model
        "prompt": "Classify the following news article into 1 of the following categories: categories: [Business, Tech, Politics, Sport, Entertainment]\n\nnews article: Donna Steffensen Is Cooking Up a New Kind of Perfection. The Internet’s most beloved cooking guru has a buzzy new book and a fresh new perspective:\n\nClassified category:",
        "max_tokens": 60,
        "temperature": 0,
        "frequency_penalty": 0,
        "presence_penalty": 0,
        "stop": "null"
    }
    endpoint = f"{deployment.deployment_url}/v2/chat-completion"

headers = {"Authorization": ai_core_client.rest_client.get_token(),
           "ai-resource-group": resource_group_id,
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=body)

print("Inference result:", response.json())
pprint(vars(response))

### 9. Kill deployment (optional)

In [ ]:
delete_resp = ai_core_client.deployment.modify(deployment_resp.id,
                                                 target_status=Status.STOPPED,
                                              resource_group=resource_group_id)
status = None
while status != Status.STOPPED:
    time.sleep(5)
    clear_output(wait=True)
    deployment = ai_core_client.deployment.get(deployment_resp.id, resource_group=resource_group_id)
    status = deployment.status
    print("...... killing deployment ......", flush=True)
    print(f"Deployment status: {deployment.status}")